In [1]:
import DifferentialEquations, DelimitedFiles, LinearAlgebra
de = DifferentialEquations
df = DelimitedFiles
la = LinearAlgebra;

In [2]:
function add_up_connections(A::Array{Float64, 2})
    pre_n = sum(convert(Array{Int}, A .!= 0); dims=2) # Number of connections from a given neuron
    pre_n[pre_n .== 0] .= 1                           # This is to remove a divide-by-zero; if n is 0, then so is A
    return pre_n
end

add_up_connections (generic function with 1 method)

In [3]:
function Θ(x::Array{Float64}, x_rev::Float64, λ::Float64, θ::Float64)
    n_cortices = length(x)
    output = Array{Float64, 2}(undef, n_cortices, n_cortices)
    for ((k, xk), (j, xj)) in Iterators.product(enumerate(x), enumerate(x))
        output[j, k] = (xj - x_rev)/(1 + exp(-λ*(xk - θ)))
    end
    return output
end

Θ (generic function with 1 method)

In [4]:
function dΘ_dx(x::Array{Float64, 1}, λ::Float64, θ::Float64)
    n_cortices = length(x)
    final = ones(Float64, n_cortices, n_cortices)/(1 + exp(-λ*(x - θ)))
    diagonal = la.diagm(0 => x*λ*exp(-λ*(x - θ))/(1 + exp(-λ*(x - θ)))^2)
    return final + diagonal
end

dΘ_dx (generic function with 1 method)

In [5]:
function hr_dots(du::Array{Float64, 2}, u::Array{Float64, 2}, p, t)
    b, i0, x_rev, λ, θ, μ, s, x_rest, α, n1, β, n2, G1, G2 = p
    x, y, z = u[1, :], u[2, :], u[3, :]
    theta = Θ(x, x_rev, λ, θ)
    du[1, :] = y - (x.^3) + b*(x.^2) + i0 - z - (α./n1).*sum(G1.*theta, dims=2) - (β./n2).*sum(G2.*theta, dims=2)
    du[2, :] = 1 .- 5*(x.^2) - y
    du[3, :] = μ*(s*(x .- x_rest) - z)
    return du
end

hr_dots (generic function with 1 method)

In [6]:
const n = df.readdlm("../connectomes/cat_connectome");

In [7]:
const N = size(n, 1)
pre_cortex_mask = zeros(Float64, N, N)
pre_cortex_mask[collect(1:17), collect(1:17)] .= 1
pre_cortex_mask[collect(17:27), collect(17:27)] .= 2
pre_cortex_mask[collect(27:45), collect(27:45)] .= 3
pre_cortex_mask[collect(45:65), collect(45:65)] .= 4
const cortex_mask = copy(pre_cortex_mask)
pre_G1 = copy(n)
pre_G1[cortex_mask .== 0] .= 0
pre_G2 = copy(n)
pre_G2[cortex_mask .!= 0] .= 0
const G1 = copy(pre_G1)
const G2 = copy(pre_G2);

In [8]:
const b = 3.2                       # Controls spiking frequency
const i0 = 4.4*ones(size(n, 2))     # Input current ---- It's an array so we can add noise later
const x_rev = 2.0                   # Reverse potential
const λ = 10.0                      # Sigmoidal function parameter
const θ = -0.25                     # Sigmoidal function parameter
const μ = 0.01                      # Time scale of slow current
const s = 4.0                       # Governs adaptation (whatever that means)
const x_rest = -1.6                 # Resting potential ------ INCORRECT IN SANTOS PAPER
α = 0.210                           # Intra connection strength ---- VARIED PARAMETER
const n1 = add_up_connections(G1)   # Intra connections
β = 0.040                           # Inter connection strength ---- VARIED PARAMETER
const n2 = add_up_connections(G2);  # Inter connections

In [9]:
p = (b, i0, x_rev, λ, θ, μ, s, x_rest, α, n1, β, n2, G1, G2)
u0 = [3.5*rand(Float64, 1, N) .- 1.5; 0.2*rand(Float64, 1, N); 0.2*rand(Float64, 1, N)];
tspan = (0.0, 4000.0);

In [10]:
prob = de.ODEProblem(hr_dots, u0, tspan, p)

ODEProblem with uType Array{Float64,2} and tType Float64. In-place: true
timespan: (0.0, 4000.0)
u0: [1.44102 -0.945258 … 0.491869 0.311321; 0.0281951 0.176508 … 0.168571 0.0851275; 0.12332 0.162566 … 0.168686 0.11392]

In [11]:
@time sol = de.solve(prob)

 48.534274 seconds (35.49 M allocations: 17.410 GiB, 10.10% gc time)


retcode: Success
Interpolation: 3rd order Hermite
t: 22729-element Array{Float64,1}:
    0.0                 
    0.003179924777774225
    0.017931446358393888
    0.04043228618851418 
    0.06692788820840682 
    0.10276056400679946 
    0.14024250757102408 
    0.1944147326057191  
    0.25514961079618853 
    0.3280156068491963  
    0.4033868388536788  
    0.4988833240846227  
    0.6010683820899738  
    ⋮                   
 3998.2171182943775     
 3998.367205662427      
 3998.5152729839456     
 3998.666372539661      
 3998.8509062763765     
 3999.0706009291616     
 3999.300932999877      
 3999.4638395519914     
 3999.6160106175553     
 3999.7974496178167     
 3999.9768312016427     
 4000.0                 
u: 22729-element Array{Array{Float64,2},1}:
 [1.44102 -0.945258 … 0.491869 0.311321; 0.0281951 0.176508 … 0.168571 0.0851275; 0.12332 0.162566 … 0.168686 0.11392]   
 [1.46695 -0.916766 … 0.509912 0.327796; -0.00227973 0.165364 … 0.167227 0.0864107; 0.123705 0.1626

In [12]:
u_f = sol.u[end]

3×65 Array{Float64,2}:
 0.502659  -0.625762  -0.617918  …  -0.858713  -0.870532  -0.920098
 0.640858  -1.60533   -1.50132      -3.09317   -3.19921   -3.67755 
 4.80934    4.39185    4.42429       4.335      4.23585    4.36961 

In [13]:
x_f = u_f[1, :]

65-element Array{Float64,1}:
  0.5026590434900133  
 -0.625762433729263   
 -0.6179179363202659  
 -0.7555613733951849  
 -0.9216684019808455  
 -1.0014953370583353  
 -0.8773668446545957  
 -0.5388591793752114  
 -0.5380389814841292  
 -0.6849909643076002  
 -0.7310921406893998  
 -0.2727765213699694  
 -0.6020860590521577  
  ⋮                   
 -0.016351000459513355
 -0.4947246047607346  
 -0.8100140032478313  
  1.010770889902242   
 -0.2360735926031848  
 -0.8755502257265427  
 -0.8682642324587444  
 -0.8558319154481235  
 -0.9258224623814277  
 -0.8587125710246578  
 -0.8705318577163343  
 -0.9200982766620768  

In [14]:
Plots.plot(x_f)

LoadError: UndefVarError: Plots not defined

In [15]:
u = sol.u;

In [16]:
u = cat(u..., dims=3)

LoadError: InterruptException:

In [17]:
Plots.plot(u[end-1000:, 1, 1])

LoadError: syntax: missing last argument in "(end - 1000):" range expression 